In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import scanpy as sc
from scipy import ndimage
import scipy as sp
import matplotlib.pylab as pl
import ot
import scipy.io
import sys
sys.path.append('../git_repos/COOT/code')
import cot
import helper_functions as fcts
from tqdm import tqdm
from sklearn import preprocessing as pp
import scib

In [3]:
import pandas as pd

In [4]:
results = []

In [5]:
for i in tqdm(['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3', 's3d6', 's3d7', 's4d1', 's4d8', 's4d9']):
    atac = sc.read('data/'+i+'/atac-emb_gc.h5ad')
    rna = sc.read('data/'+i+'/rna-emb_gc.h5ad')
    xs = pp.normalize(rna.obsm['X_pca'])
    xt = pp.normalize(atac.obsm['X_lsi_red']) 
    
    for eps_1 in tqdm([1e-3,.5e-2,1e-2,.5e-1,1e-1, .5]):
        for eps_2 in tqdm([1e-3,.5e-2,1e-2,.5e-1,1e-1, .5]):
            ot.tic()
            Ts,Tv,_,log=cot.cot_numpy(xs,xt,niter=50,algo = 'sinkhorn', algo2='sinkhorn', reg=eps_1, reg2=eps_2, log=True)
            ot.toc()
        
            translated = Ts.dot(xt)/ np.sum(Ts,axis=1)[:,None]

            adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA (translated)", "ATAC"])
            adata.obsm["X_coot"] = np.concatenate((translated, xt), axis=0)

            # Evaluation metrics
            f = fcts.foscttm(translated, xt)
            lta = fcts.transfer_accuracy(translated, xt, atac.obs['cell_type'], rna.obs['cell_type'])
            alg_score = fcts.alignment_score(np.array(translated), np.array(xt))
            rmse = fcts.calculate_rmse(translated, xt)

            # Bio-conservation metrics
            sc.pp.neighbors(adata, use_rep='X_coot')
            scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
            nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
            asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_coot')
            ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

            # Batch correction metrics
            asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_coot',verbose=False)
            graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')

            results.append([i, eps_1, eps_2, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])
            df_results = pd.DataFrame(
                results,
                columns = ["donor", "epsilon_1", "epsilon_2", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
            )
            df_results.to_csv('results_coot.csv', index = False)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Delta: 0.011696845662163663  Loss: 0.07362875362759039
Delta: 0.039891321552907706  Loss: 0.06869377754288088
Delta: 0.0388498443070155  Loss: 0.06214612306579361
Delta: 0.03701186471731894  Loss: 0.05878116490456542
Delta: 0.035605770382223664  Loss: 0.05634434472696123
Delta: 0.039502082310292216  Loss: 0.054463380167367584
Delta: 0.02823241258016003  Loss: 0.05272028860452946
Delta: 0.01961662844895235  Loss: 0.05195712838445244
Delta: 0.014914449062071452  Loss: 0.05160091969127891
Delta: 0.011581821035210715  Loss: 0.05138429951762746
Delta: 0.010194738736840993  Loss: 0.05124445196954288
Delta: 0.00371298877035401  Loss: 0.051175979876972436
Delta: 0.001480588701616692  Loss: 0.051167200986166395
Delta: 0.0010198405042879423  Loss: 0.05116668326281958
Delta: 0.0007562305506588026  Loss: 0.051166132483772504
Delta: 0.0005406349378782083  Loss: 0.05116545610877894
Delta: 0.00037962125629008406  Loss: 0.05116479414647797
Delta: 0.00027251274233403683  Loss: 0.051164214152290825
Delt

/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3530284/1559086866.py:26: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


 17%|█▋        | 1/6 [04:53<24:25, 293.16s/it]

Delta: 0.0027533655740806977  Loss: 0.07371071442806136
Delta: 0.01653452124378257  Loss: 0.07263426753646998
Delta: 0.02584212927133947  Loss: 0.06752699243174447
Delta: 0.01948897029898547  Loss: 0.06426237103565767
Delta: 0.01754140900454921  Loss: 0.06241718116439755
Delta: 0.004286981258661213  Loss: 0.06141849223193675
Delta: 0.004518909162881424  Loss: 0.06127182342505449
Delta: 0.009189208655005636  Loss: 0.06097236152668484
Delta: 0.00878318316720742  Loss: 0.06041064932690994
Delta: 0.009682808807684452  Loss: 0.05991770688193128
Delta: 0.007022363068258361  Loss: 0.05942926452618041
Delta: 0.0014919607691609733  Loss: 0.05925711258593609
Delta: 0.0007106018143967644  Loss: 0.05922488284540763
Delta: 0.000499160385075834  Loss: 0.0592107822776451
Delta: 0.00045935812438468624  Loss: 0.059200758723312014
Delta: 0.0005402098759945407  Loss: 0.059191391315230824
Delta: 0.0007285959606194581  Loss: 0.05918009395211246
Delta: 0.001074287866971122  Loss: 0.059163162740271696
Delta:

/tmp/ipykernel_3530284/1559086866.py:26: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


 33%|███▎      | 2/6 [12:29<25:55, 388.92s/it]

...

/tmp/ipykernel_3530284/1559086866.py:26: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


 67%|██████▋   | 4/6 [03:01<01:30, 45.27s/it]

Delta: 8.283590960369255e-06  Loss: 0.0755465659306802
Delta: 2.751947162337222e-09  Loss: 0.07554656588132039
converged at iter  1
Elapsed time : 3.0167477130889893 s


/tmp/ipykernel_3530284/1559086866.py:26: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


 83%|████████▎ | 5/6 [03:44<00:44, 44.60s/it]

Delta: 2.2421986766537165e-06  Loss: 0.07554657004019627
Delta: 1.5405377558299128e-10  Loss: 0.07554657003077281
converged at iter  1
Elapsed time : 2.92742657661438 s


/tmp/ipykernel_3530284/1559086866.py:26: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


100%|██████████| 6/6 [04:28<00:00, 44.83s/it]

100%|██████████| 5/5 [3:22:09<00:00, 2425.87s/it]


In [9]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "epsilon_1","epsilon_2", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)
df_results

,donor,epsilon_1,epsilon_2,foscttm,label_transfer_accuracy,alignment_score,rmse,nmi,asw_label,ari,asw_batch,graph_connectivity
0,s1d1,0.001,0.001,0.3759,0.306395,0.853630,0.275386,0.462986,0.497683,0.370139,0.790501,0.749880
1,s1d1,0.001,0.005,0.3647,0.275868,0.830658,0.258202,0.504794,0.505869,0.405349,0.691036,0.740942
2,s1d1,0.001,0.010,0.4892,0.081941,0.801822,0.263598,0.441205,0.499662,0.264707,0.632119,0.722633
3,s1d1,0.001,0.050,0.4599,0.112789,0.551727,0.247105,0.493765,0.495997,0.271899,0.532512,0.608115
4,s1d1,0.001,0.100,0.4602,0.173522,0.217355,0.238604,0.433398,0.492608,0.269628,0.495688,0.435200
...,...,...,...,...,...,...,...,...,...,...,...,...
175,s2d4,0.500,0.005,0.5099,0.063001,0.109601,0.235634,0.284054,0.500606,0.213107,0.406894,0.407509
176,s2d4,0.500,0.010,0.5101,0.063001,0.109602,0.235634,0.278308,0.500606,0.210832,0.406894,0.401572
177,s2d4,0.500,0.050,0.5103,0.063001,0.109601,0.235634,0.278028,0.500606,0.208791,0.406894,0.401198
178,s2d4,0.500,0.100,0.5103,0.063001,0.109601,0.235634,0.281382,0.500606,0.213653,0.406894,0.403340
